In [ ]:
import numpy as np
import cv2
import os
import open3d as o3d
import time
from scipy.spatial.transform import Rotation as R_scipy
import g2o

In [3]:
def read_associations(filepath):
    """Reads the associations.txt file."""
    associations = []
    with open(filepath, 'r') as f:
        for line in f:
            if line.strip() and not line.startswith('#'):
                parts = line.strip().split()
                if len(parts) == 4:
                    associations.append((float(parts[0]), parts[1], float(parts[2]), parts[3]))
    return associations

def read_groundtruth(filepath):
    """Reads the groundtruth.txt file."""
    groundtruth = {}
    with open(filepath, 'r') as f:
        for line in f:
            if line.strip() and not line.startswith('#'):
                parts = line.strip().split()
                if len(parts) == 8:
                    timestamp = float(parts[0])
                    pose = np.array(list(map(float, parts[1:])))
                    groundtruth[timestamp] = pose
    return groundtruth

In [4]:
def load_rgbd_frame(dataset_path, rgb_filename, depth_filename):
    """Loads a synchronized RGB and Depth image pair."""
    rgb_path = os.path.join(dataset_path, rgb_filename)
    depth_path = os.path.join(dataset_path, depth_filename)

    rgb_image = cv2.imread(rgb_path, cv2.IMREAD_COLOR)
    depth_image_raw = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)

    if rgb_image is None or depth_image_raw is None:
        print(f"Error loading images: RGB={rgb_path}, Depth={depth_path}")
        return None, None

    depth_image_meters = depth_image_raw.astype(np.float32) / 5000.0
    depth_image_meters[depth_image_meters == 0] = np.nan

    return rgb_image, depth_image_meters

In [5]:
def rgbd_to_point_cloud(rgb_image, depth_image_meters, camera_matrix):
    """Converts an RGB-D image pair into a colored 3D point cloud."""
    height, width, _ = rgb_image.shape
    fx, fy = camera_matrix[0, 0], camera_matrix[1, 1]
    cx, cy = camera_matrix[0, 2], camera_matrix[1, 2]

    u, v = np.meshgrid(np.arange(width), np.arange(height))
    u = u.astype(np.float32)
    v = v.astype(np.float32)

    Z = depth_image_meters.copy()
    valid_mask = ~np.isnan(Z)

    X = (u - cx) * Z / fx
    Y = (v - cy) * Z / fy

    points_3d = np.stack((X, Y, Z), axis=-1).reshape(-1, 3)
    colors = rgb_image.reshape(-1, 3)

    points_3d = points_3d[valid_mask.flatten()]
    colors = colors[valid_mask.flatten()]

    colors = colors[:, [2, 1, 0]]
    return points_3d, colors / 255.0

In [ ]:
class MapPoint:
    """Represents a 3D point in the map."""
    def __init__(self, id, position_3d):
        self.id = id
        self.position_3d = np.array(position_3d, dtype=np.float64)
        # Observations: {keyframe_id: (feature_idx_in_kf, descriptor)}
        # This links which 2D feature in which keyframe observes this 3D map point.
        self.observations = {} 

    def add_observation(self, keyframe_id, feature_idx, descriptor):
        self.observations[keyframe_id] = (feature_idx, descriptor)

    def get_descriptor(self):
        if self.observations:
            for obs_data in self.observations.values():
                return obs_data[1] 
        return None

In [ ]:
class KeyFrame:
    """Represents a keyframe in the map."""
    def __init__(self, id, timestamp, pose, kps, des, rgb_image_path=None, depth_image_path=None):
        self.id = id
        self.timestamp = timestamp
        self.pose = np.array(pose, dtype=np.float64) # 4x4 SE(3) pose matrix (World_T_Camera)
        self.kps = kps # List of cv2.KeyPoint objects
        self.des = des # NumPy array of descriptors
        # Observed_map_points: {feature_idx_in_kf: map_point_id}
        # This links a 2D feature in this keyframe to its observed 3D map point.
        self.observed_map_points = {} 
        self.rgb_image_path = rgb_image_path
        self.depth_image_path = depth_image_path

    def add_map_point_observation(self, feature_idx, map_point_id):
        self.observed_map_points[feature_idx] = map_point_id

In [ ]:
class Map:
    """Manages all KeyFrames and MapPoints."""
    def __init__(self):
        self.keyframes = {} # {keyframe_id: KeyFrame_object}
        self.map_points = {} # {mappoint_id: MapPoint_object}
        self.next_keyframe_id = 0
        self.next_mappoint_id = 0

    def add_keyframe(self, keyframe_data):
        keyframe_id = self.next_keyframe_id
        kf = KeyFrame(keyframe_id, **keyframe_data)
        self.keyframes[keyframe_id] = kf
        self.next_keyframe_id += 1
        return kf

    def add_mappoint(self, position_3d):
        mappoint_id = self.next_mappoint_id
        mp = MapPoint(mappoint_id, position_3d)
        self.map_points[mappoint_id] = mp
        self.next_mappoint_id += 1
        return mp

    # ...existing code...
    def local_bundle_adjustment(self, camera_matrix, current_keyframe_id):
        """
        Bundle Adjustment on a local window of keyframes and their observed map points,
        using miquelmassot/g2o-python API.
        """
        if g2o is None:
            print("g2o-python not installed. Skipping Local Bundle Adjustment.")
            return

        print(f"INFO: Performing Local Bundle Adjustment around KeyFrame {current_keyframe_id}...")

        optimizer = g2o.SparseOptimizer()
        solver = g2o.BlockSolverX(g2o.LinearSolverDenseX())
        algorithm = g2o.OptimizationAlgorithmLevenberg(solver)
        optimizer.set_algorithm(algorithm)

        fx, fy = camera_matrix[0, 0], camera_matrix[1, 1]
        cx, cy = camera_matrix[0, 2], camera_matrix[1, 2]

        # Select last 10 keyframes up to current
        kf_ids_in_window = sorted([kf_id for kf_id in self.keyframes.keys() if kf_id <= current_keyframe_id])[-10:]
        if not kf_ids_in_window:
            return

        keyframes_to_optimize = [self.keyframes[kf_id] for kf_id in kf_ids_in_window]

        # Add keyframe pose vertices
        for kf in keyframes_to_optimize:
            v_se3 = g2o.VertexSE3Expmap()
            v_se3.set_estimate(g2o.SE3Quat(kf.pose[:3, :3], kf.pose[:3, 3]))
            v_se3.set_id(kf.id)
            optimizer.add_vertex(v_se3)
            if kf.id == kf_ids_in_window[0]:
                v_se3.set_fixed(True)

        # Collect all map points observed in these keyframes
        mappoints_to_optimize = set()
        for kf in keyframes_to_optimize:
            for mp_id in kf.observed_map_points.values():
                mappoints_to_optimize.add(mp_id)

        # Add map point vertices
        for mp_id in mappoints_to_optimize:
            mp = self.map_points[mp_id]
            v_point = g2o.VertexSBAPointXYZ()
            v_point.set_estimate(mp.position_3d)
            v_point.set_id(mp.id + 100000)
            v_point.set_marginalized(True)
            optimizer.add_vertex(v_point)

        # Add edges (observations)
        for kf in keyframes_to_optimize:
            v_se3 = optimizer.vertex(kf.id)
            for feat_idx, mp_id in kf.observed_map_points.items():
                if mp_id in mappoints_to_optimize:
                    mp = self.map_points[mp_id]
                    v_point = optimizer.vertex(mp.id + 100000)
                    edge = g2o.EdgeProjectXYZ2UV()
                    edge.set_vertex(0, v_point)
                    edge.set_vertex(1, v_se3)
                    observed_uv = kf.kps[feat_idx].pt
                    edge.set_measurement(observed_uv)
                    edge.set_camera_parameters(fx, fy, cx, cy)
                    information_matrix = np.eye(2) * (1.0 / (1.0**2))
                    edge.set_information(information_matrix)
                    robust_kernel = g2o.RobustKernelHuber()
                    robust_kernel.set_delta(1.0)
                    edge.set_robust_kernel(robust_kernel)
                    optimizer.add_edge(edge)

        optimizer.initialize_optimization()
        optimizer.optimize(20)

        # Update keyframes
        for kf_id in kf_ids_in_window:
            v_se3 = optimizer.vertex(kf_id)
            if v_se3:
                optimized_pose_quat = v_se3.estimate()
                optimized_pose = optimized_pose_quat.matrix()
                self.keyframes[kf_id].pose = optimized_pose

        # Update map points
        for mp_id in mappoints_to_optimize:
            v_point = optimizer.vertex(mp_id + 100000)
            if v_point:
                optimized_position = v_point.estimate()
                self.map_points[mp_id].position_3d = optimized_position

        print(f"INFO: Local BA completed. Optimized {len(kf_ids_in_window)} KeyFrames and {len(mappoints_to_optimize)} MapPoints.")
    # ...existing code...

    def global_bundle_adjustment(self, camera_matrix):
        """
        Conceptual placeholder for Global Bundle Adjustment.
        Similar to local BA but operates on all keyframes and map points.
        """
        if g2o is None:
            print("g2o-python not installed. Skipping Global Bundle Adjustment.")
            return
        print("INFO: Performing Global Bundle Adjustment (Conceptual)...")
        # Full implementation would be similar to local_bundle_adjustment
        # but iterating over all keyframes and map points in self.keyframes/self.map_points
        # and would typically be very slow.
        pass

    def pose_graph_optimization(self, loop_constraint_kf_id, loop_constraint_transform):
        """
        Optimizes the pose graph after a loop closure is detected,
        using miquelmassot/g2o-python API.
        `loop_constraint_transform` should be the 4x4 relative transformation
        (e.g., T_current_kf_loop_kf) between the current keyframe and the detected loop keyframe.
        """
        if g2o is None:
            print("g2o-python not installed. Skipping Pose Graph Optimization.")
            return

        print("INFO: Performing Pose Graph Optimization...")

        optimizer = g2o.SparseOptimizer()
        # Use BlockSolverSE3 for SE3 vertices/edges in PGO
        solver = g2o.BlockSolverSE3(g2o.LinearSolverDenseSE3()) # LinearSolverDenseSE3 for 6x6 blocks
        algorithm = g2o.OptimizationAlgorithmLevenberg(solver)
        optimizer.set_algorithm(algorithm)

        # Add all Keyframe Poses as Vertices (VertexSE3)
        for kf_id, kf in self.keyframes.items():
            v_se3 = g2o.VertexSE3() # VertexSE3 is fine for PGO (not BA, which uses Expmap)
            v_se3.set_id(kf.id)
            v_se3.set_estimate(g2o.SE3Quat(kf.pose[:3,:3], kf.pose[:3,3]))
            optimizer.add_vertex(v_se3)

            # Fix the first keyframe to anchor the graph
            if kf.id == 0:
                v_se3.set_fixed(True)

        # Add Odometry Edges (relative transformations between consecutive KeyFrames)
        kf_ids_sorted = sorted(self.keyframes.keys())
        for i in range(len(kf_ids_sorted) - 1):
            kf1_id = kf_ids_sorted[i]
            kf2_id = kf_ids_sorted[i+1]
            kf1 = self.keyframes[kf1_id]
            kf2 = self.keyframes[kf2_id]

            # Relative transform from kf1 to kf2: T_kf1_kf2 = T_kf1_W @ T_W_kf2 = inv(T_W_kf1) @ T_W_kf2
            T_kf1_kf2 = np.linalg.inv(kf1.pose) @ kf2.pose

            edge = g2o.EdgeSE3() # EdgeSE3 for 6D pose constraints
            edge.set_vertex(0, optimizer.vertex(kf1_id))
            edge.set_vertex(1, optimizer.vertex(kf2_id))
            edge.set_measurement(g2o.SE3Quat(T_kf1_kf2[:3,:3], T_kf1_kf2[:3,3]))
            # Information matrix for odometry (high confidence)
            edge.set_information(np.eye(6) * 100.0) # Example: High confidence
            optimizer.add_edge(edge)

        # Add the Loop Closure Edge
        current_kf = self.keyframes.get(self.next_keyframe_id -1) # Latest keyframe
        loop_kf = self.keyframes.get(loop_constraint_kf_id) # The keyframe we looped to

        if current_kf and loop_kf and loop_constraint_transform is not None:
            loop_edge = g2o.EdgeSE3()
            loop_edge.set_vertex(0, optimizer.vertex(loop_kf.id)) # Loop Keyframe
            loop_edge.set_vertex(1, optimizer.vertex(current_kf.id)) # Current Keyframe
            loop_edge.set_measurement(g2o.SE3Quat(loop_constraint_transform[:3,:3], loop_constraint_transform[:3,3]))
            # Information matrix for loop closure (usually lower confidence than odometry, but critical)
            loop_edge.set_information(np.eye(6) * 10.0) # Example: Medium confidence
            optimizer.add_edge(loop_edge)
            print(f"Added loop closure edge between KF {loop_kf.id} and KF {current_kf.id}.")

        # Perform Optimization
        optimizer.initialize_optimization()
        optimizer.optimize(50) # More iterations for global optimization

        # Update Map with Optimized Values
        for kf_id, kf in self.keyframes.items():
            v_se3 = optimizer.vertex(kf_id)
            if v_se3:
                optimized_pose_quat = v_se3.estimate()
                optimized_pose = optimized_pose_quat.matrix()
                kf.pose = optimized_pose
        
        print("INFO: Pose Graph Optimization completed.")
        # After PGO, you would typically trigger a Global Bundle Adjustment
        # or re-triangulate/refine map points based on the new keyframe poses.

In [ ]:
class ORBSLAMSystem:
    def __init__(self, camera_matrix, dist_coeffs):
        self.camera_matrix = camera_matrix
        self.dist_coeffs = dist_coeffs

        self.orb = cv2.ORB_create(nfeatures=2000, scaleFactor=1.2, nlevels=8)
        self.bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

        self.map = Map() # Central map object
        self.current_estimated_pose = np.eye(4) # Initial pose (World_T_Camera)
        self.estimated_trajectory_poses = [] # List of 4x4 poses

        self.last_frame_kps = None
        self.last_frame_des = None
        self.last_keyframe = None # Reference to the last added KeyFrame object

        self.frame_count = 0

    def process_frame(self, timestamp, rgb_image, depth_image_meters):
        self.frame_count += 1
        rgb_image_processed = rgb_image.copy()
        rgb_image_gray = cv2.cvtColor(rgb_image_processed, cv2.COLOR_BGR2GRAY)

        current_kps, current_des = self.orb.detectAndCompute(rgb_image_gray, None)

        if current_des is None or len(current_kps) < 5:
            print(f"Warning: Not enough features detected ({len(current_kps)}). Skipping pose estimation.")
            if len(self.estimated_trajectory_poses) > 0:
                self.estimated_trajectory_poses.append(self.current_estimated_pose)
            return self.current_estimated_pose, self._get_current_map_point_cloud()

        if self.last_keyframe is None: # First frame initialization
            self.current_estimated_pose = np.eye(4) # Start at origin
            self.estimated_trajectory_poses.append(self.current_estimated_pose)

            # Add initial keyframe to map
            self.last_keyframe = self.map.add_keyframe({
                'timestamp': timestamp,
                'pose': self.current_estimated_pose,
                'kps': current_kps,
                'des': current_des,
            })

            # Initialize map points from the first keyframe using depth
            points_3d, colors = rgbd_to_point_cloud(rgb_image_processed, depth_image_meters, self.camera_matrix)
            
            # --- IMPORTANT: Associate 2D features with 3D MapPoints during initialization ---
            # This part needs to be robustly implemented for real BA!
            # For simplicity, we create a MapPoint for each valid 3D point and associate it with
            # the corresponding 2D feature IF a feature exists at that pixel location,
            # or simply with a feature by index for demonstration.
            # In a real system:
            # 1. Project 3D points from depth map to 2D.
            # 2. Find ORB features close to these projected points.
            # 3. Create MapPoints only for those ORB features that have valid depth.
            
            # A more robust initialization would use PnP or Bundle Adjustment on initial frames.
            # Here, we'll try a basic association for existing kps that 'fall into' valid 3D points
            
            # Create a KD-tree or similar for efficient nearest neighbor search if many kps
            kp_coords_2d = np.array([kp.pt for kp in current_kps])
            
            # Instead of iterating through all 3D points, let's iterate through detected features
            # and find their 3D point from depth map (if available).
            for feat_idx, kp in enumerate(current_kps):
                u, v = int(round(kp.pt[0])), int(round(kp.pt[1]))
                if 0 <= v < depth_image_meters.shape[0] and 0 <= u < depth_image_meters.shape[1]:
                    depth_val = depth_image_meters[v, u]
                    if not np.isnan(depth_val) and depth_val > 0:
                        # Convert 2D point (u,v) with depth to 3D point in camera frame
                        X_c = (u - self.camera_matrix[0, 2]) * depth_val / self.camera_matrix[0, 0]
                        Y_c = (v - self.camera_matrix[1, 2]) * depth_val / self.camera_matrix[1, 1]
                        Z_c = depth_val
                        
                        p3d_camera_frame = np.array([X_c, Y_c, Z_c, 1.0])
                        # Transform to world frame (assuming current_estimated_pose is identity for first frame)
                        p3d_world = self.current_estimated_pose @ p3d_camera_frame
                        p3d_world = p3d_world[:3] # Convert homogeneous to 3D

                        mp = self.map.add_mappoint(p3d_world)
                        descriptor = current_des[feat_idx] if current_des is not None else None
                        mp.add_observation(self.last_keyframe.id, feat_idx, descriptor)
                        self.last_keyframe.add_map_point_observation(feat_idx, mp.id)

            print(f"Initialized with {len(self.map.map_points)} map points and KeyFrame {self.last_keyframe.id}.")

            self.last_frame_kps = current_kps
            self.last_frame_des = current_des
            return self.current_estimated_pose, self._get_current_map_point_cloud()

        # Tracking: Match features with last frame
        if self.last_frame_des is None or current_des.dtype != self.last_frame_des.dtype:
            print("Descriptor type mismatch or last_frame_des is None. Resetting tracking.")
            self.last_frame_kps = current_kps
            self.last_frame_des = current_des
            if len(self.estimated_trajectory_poses) > 0:
                self.estimated_trajectory_poses.append(self.current_estimated_pose)
            return self.current_estimated_pose, self._get_current_map_point_cloud()

        matches = self.bf.match(current_des, self.last_frame_des)
        matches = sorted(matches, key=lambda x: x.distance)
        good_matches = matches[:200]

        if len(good_matches) < 8:
            print(f"Warning: Not enough good matches ({len(good_matches)}) for pose estimation. Using last pose.")
            self.estimated_trajectory_poses.append(self.current_estimated_pose)
            self.last_frame_kps = current_kps
            self.last_frame_des = current_des
            return self.current_estimated_pose, self._get_current_map_point_cloud()

        pts_curr = np.float32([current_kps[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        pts_prev = np.float32([self.last_frame_kps[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        E, mask = cv2.findEssentialMat(pts_curr, pts_prev, self.camera_matrix, method=cv2.RANSAC, prob=0.999, threshold=1.0)
        _, R_prev_curr, t_prev_curr, mask_RP = cv2.recoverPose(E, pts_curr, pts_prev, self.camera_matrix, mask=mask)

        T_prev_curr = np.eye(4)
        T_prev_curr[:3, :3] = R_prev_curr
        T_prev_curr[:3, 3] = t_prev_curr.flatten()

        T_C_prev_C_curr = np.linalg.inv(T_prev_curr) # Transformation from previous to current camera
        self.current_estimated_pose = self.current_estimated_pose @ T_C_prev_C_curr
        self.estimated_trajectory_poses.append(self.current_estimated_pose)

        # Keyframe Management and Map Point Triangulation
        translation_diff = np.linalg.norm(self.current_estimated_pose[:3, 3] - self.last_keyframe.pose[:3, 3])
        
        is_keyframe = False
        if translation_diff > 0.15 or (self.frame_count % 30 == 0 and len(self.map.keyframes) > 1):
            is_keyframe = True
            
            new_keyframe = self.map.add_keyframe({
                'timestamp': timestamp,
                'pose': self.current_estimated_pose,
                'kps': current_kps,
                'des': current_des,
            })
            print(f"Added new KeyFrame {new_keyframe.id} at frame {self.frame_count}.")

            good_matches_inliers = [m for m, mask_val in zip(good_matches, mask_RP.flatten()) if mask_val == 1]
            
            if len(good_matches_inliers) >= 8:
                pts_kf1_matched = np.float32([self.last_keyframe.kps[m.trainIdx].pt for m in good_matches_inliers]).reshape(-1, 2)
                pts_kf2_matched = np.float32([current_kps[m.queryIdx].pt for m in good_matches_inliers]).reshape(-1, 2)

                P_kf1 = self.camera_matrix @ self.last_keyframe.pose[:3, :]
                P_curr = self.camera_matrix @ new_keyframe.pose[:3, :]

                points_4d_hom = cv2.triangulatePoints(P_kf1, P_curr, pts_kf1_matched.T, pts_kf2_matched.T)
                points_3d_new_world = (points_4d_hom / points_4d_hom[3]).T[:, :3]

                valid_triangulation_mask = (points_3d_new_world[:, 2] > 0) # Basic depth filter (positive Z)
                points_3d_new_world = points_3d_new_world[valid_triangulation_mask]

                # --- IMPORTANT: Associate 2D features with new 3D MapPoints ---
                valid_matches_for_new_points = [m for m, valid in zip(good_matches_inliers, valid_triangulation_mask) if valid]

                if points_3d_new_world.shape[0] > 0:
                    for j, p3d in enumerate(points_3d_new_world):
                        match_info = valid_matches_for_new_points[j]
                        
                        mp = self.map.add_mappoint(p3d)
                        
                        # Add observation from the previous keyframe (trainer of the match)
                        mp_descriptor = self.last_keyframe.des[match_info.trainIdx] if self.last_keyframe.des is not None else None
                        mp.add_observation(self.last_keyframe.id, match_info.trainIdx, mp_descriptor)
                        self.last_keyframe.add_map_point_observation(match_info.trainIdx, mp.id)

                        # Add observation from the current keyframe (query of the match)
                        mp_descriptor_curr = new_keyframe.des[match_info.queryIdx] if new_keyframe.des is not None else None
                        mp.add_observation(new_keyframe.id, match_info.queryIdx, mp_descriptor_curr)
                        new_keyframe.add_map_point_observation(match_info.queryIdx, mp.id)
                        
                    print(f"Triangulated {points_3d_new_world.shape[0]} new map points and linked observations.")

            # --- LOCAL BUNDLE ADJUSTMENT ---
            self.map.local_bundle_adjustment(self.camera_matrix, new_keyframe.id)

            # --- LOOP CLOSURE (Conceptual Placeholder) ---
            # Simulate a detected loop every N frames and trigger PGO.
            if new_keyframe.id > 0 and new_keyframe.id % 10 == 0: # Example: Check every 10th keyframe
                # In a real system:
                # 1. Place Recognition: Use DBoW2 or similar to find a loop candidate (e.g., matching KF 0)
                # 2. Geometric Verification: Verify the loop geometrically (e.g., with RANSAC-based PnP)
                # 3. If verified, obtain the relative transformation (T_current_kf_loop_kf)
                
                # Simulating a loop to KF 0 (first keyframe)
                print(f"Simulating loop closure: KF {new_keyframe.id} potentially loops to KF 0.")
                if len(self.map.keyframes) > 0:
                    loop_kf_id = 0 # Assume we loop back to the very first keyframe for simulation
                    loop_kf_obj = self.map.keyframes.get(loop_kf_id)
                    if loop_kf_obj:
                        # T_current_kf_loop_kf = inv(T_W_curr_kf) @ T_W_loop_kf
                        simulated_loop_transform = np.linalg.inv(new_keyframe.pose) @ loop_kf_obj.pose
                        
                        self.map.pose_graph_optimization(loop_kf_id, simulated_loop_transform)
                        # After PGO, it's common to run a Global BA to further refine the map
                        # self.map.global_bundle_adjustment(self.camera_matrix)
                    else:
                        print(f"Simulated loop target KeyFrame {loop_kf_id} not found.")
            
            self.last_keyframe = new_keyframe

        self.last_frame_kps = current_kps
        self.last_frame_des = current_des

        return self.current_estimated_pose, self._get_current_map_point_cloud()

    def _get_current_map_point_cloud(self):
        """Generates an Open3D PointCloud from the current MapPoints."""
        points = []
        colors = []
        if self.map.map_points:
            for mp_id, mp in self.map.map_points.items():
                points.append(mp.position_3d)
                colors.append([0.0, 0.7, 1.0]) # Light blue for map points
        
        o3d_pc = o3d.geometry.PointCloud()
        if points:
            o3d_pc.points = o3d.utility.Vector3dVector(np.array(points))
            o3d_pc.colors = o3d.utility.Vector3dVector(np.array(colors))
        return o3d_pc

In [6]:
fx = 525.0
fy = 525.0
cx = 319.5
cy = 239.5
k1 = 0.0  # Example value, adjust if needed
k2 = 0.0  # Example value, adjust if needed
p1 = 0.0  # Example value, adjust if needed
p2 = 0.0  # Example value, adjust if needed
k3 = 0.0  # Example value, adjust if needed

camera_matrix = np.array([[fx, 0, cx],
                          [0, fy, cy],
                          [0, 0, 1]], dtype=np.float32)

dist_coeffs = np.array([k1, k2, p1, p2, k3], dtype=np.float32)

In [7]:
dataset_path = r'rgbd_dataset_freiburg3_long_office_household' # Replace with your actual path!
associations_file = os.path.join(dataset_path, 'associations.txt')
groundtruth_file = os.path.join(dataset_path, 'groundtruth.txt')

associations = read_associations(associations_file)
groundtruth_poses = read_groundtruth(groundtruth_file)

print(f"Loaded {len(associations)} associated RGB-D frames.")
print(f"Loaded {len(groundtruth_poses)} ground truth poses.")


Loaded 2488 associated RGB-D frames.
Loaded 8710 ground truth poses.


In [ ]:
slam_system = ORBSLAMSystem(camera_matrix, dist_coeffs)

# --- Open3D Visualization Setup ---
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="ORB-SLAM (Conceptual with g2o-python)", width=1280, height=720)
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])
vis.add_geometry(mesh_frame)

# Estimated Trajectory LineSet (will be updated)
estimated_trajectory_line_set = o3d.geometry.LineSet()
vis.add_geometry(estimated_trajectory_line_set)

# Ground Truth Trajectory LineSet (will be updated)
ground_truth_trajectory_line_set = o3d.geometry.LineSet()
vis.add_geometry(ground_truth_trajectory_line_set)

# Global Map Point Cloud (will be updated by the SLAM system)
global_map_cloud_o3d = o3d.geometry.PointCloud()
vis.add_geometry(global_map_cloud_o3d)


view_control = vis.get_view_control()
view_control.set_lookat([0, 0, 1])
view_control.set_up([0, -1, 0])
view_control.set_front([0, 0, -1])

# Lists to store points for trajectory lines
ground_truth_points = []
estimated_points = []

# --- Main Loop to Process Frames ---
for i, (rgb_ts, rgb_file, depth_ts, depth_file) in enumerate(associations):
    # Process every Nth frame for a quicker demo or all frames for full processing
    if i % 5 == 0: # Process every 5th frame for more detail
        print(f"Processing frame {i}...")

        rgb_image, depth_image_meters = load_rgbd_frame(dataset_path, rgb_file, depth_file)

        if rgb_image is not None and depth_image_meters is not None:
            # --- Process frame with the SLAM system ---
            estimated_pose_current_frame, current_map_cloud_from_slam = slam_system.process_frame(rgb_ts, rgb_image, depth_image_meters)

            # --- Update Ground Truth Trajectory in visualization ---
            closest_gt_ts = min(groundtruth_poses.keys(), key=lambda ts: abs(ts - rgb_ts))
            if abs(closest_gt_ts - rgb_ts) < 0.05: # Threshold for association
                gt_pose_data = groundtruth_poses[closest_gt_ts]
                tx, ty, tz = gt_pose_data[0], gt_pose_data[1], gt_pose_data[2]
                qx, qy, qz, qw = gt_pose_data[3], gt_pose_data[4], gt_pose_data[5], gt_pose_data[6]

                R_gt = o3d.geometry.get_rotation_matrix_from_quaternion([qw, qx, qy, qz])
                T_gt = np.eye(4)
                T_gt[:3, :3] = R_gt
                T_gt[:3, 3] = np.array([tx, ty, tz])

                ground_truth_points.append(T_gt[:3, 3]) # Append position from ground truth
                if len(ground_truth_points) > 1:
                    gt_lines = [[len(ground_truth_points) - 2, len(ground_truth_points) - 1]]
                    ground_truth_trajectory_line_set.points = o3d.utility.Vector3dVector(ground_truth_points)
                    ground_truth_trajectory_line_set.lines = o3d.utility.Vector2iVector(gt_lines)
                    ground_truth_trajectory_line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0]]) # Red for ground truth
                    vis.update_geometry(ground_truth_trajectory_line_set)


            # --- Update Estimated Trajectory in visualization ---
            estimated_points.append(estimated_pose_current_frame[:3, 3]) # Append position from estimated pose
            if len(estimated_points) > 1:
                est_lines = [[len(estimated_points) - 2, len(estimated_points) - 1]]
                estimated_trajectory_line_set.points = o3d.utility.Vector3dVector(estimated_points)
                estimated_trajectory_line_set.lines = o3d.utility.Vector2iVector(est_lines)
                estimated_trajectory_line_set.colors = o3d.utility.Vector3dVector([[0, 0, 1]]) # Blue for estimated
                vis.update_geometry(estimated_trajectory_line_set)

            # --- Update Map Point Cloud in visualization ---
            global_map_cloud_o3d.points = current_map_cloud_from_slam.points
            global_map_cloud_o3d.colors = current_map_cloud_from_slam.colors
            vis.update_geometry(global_map_cloud_o3d)


            # --- Render the scene ---
            vis.poll_events()
            vis.update_renderer()
            # time.sleep(0.01) # Optional: Add a small delay for smoother visualization
        else:
            print(f"Skipping frame {i} due to image loading error.")

print("Processing finished. Close the Open3D window to exit.")
vis.destroy_window()


[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
Processing frame 0...
Processing frame 10...
Processing frame 20...
Processing frame 30...
Triangulated 36 new map points.
Processing frame 40...
Triangulated 188 new map points.
Processing frame 50...
Triangulated 17 new map points.
Processing frame 60...
Triangulated 28 new map points.
Processing frame 70...
Triangulated 115 new map points.
Processing frame 80...
Triangulated 188 new map points.
Processing frame 90...
Triangulated 157 new map points.
Processing frame 100...
Triangulated 121 new map points.
Processing frame 110...
Triangulated 71 new map points.
Processing frame 120...
Triangulated 175 new map points.
Processing frame 130...
Triangulated 165 new map points.
Processing frame 140...
Triangulated 182 new map points.
Process

In [9]:
import open3d as o3d
print(o3d.__version__)

0.19.0
